In [1]:
import graphlab

In [2]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 
                  'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

# Write the function for wordcount

In [3]:
def advWordCount(wordCount, selectedWords):
    words = wordCount.keys()
    newWordCount = {}
    
    for key in selectedWords:
        if key in wordCount:
            newWordCount[key] = wordCount[key]

    return newWordCount

### Test function

In [4]:
test_data = {
 'had': 1,
 'hands': 1,
 'happy': 1,
 'has': 5,
 'have': 1,
 'head': 1,
 'hear': 1,
 'helps': 1,
 'her': 1,
 'hold': 1,
 'hooves': 1,
 'horns': 1,
 'amazing': 1}

print advWordCount(test_data, selected_words)

{'amazing': 1}


# Import data

In [5]:
products = graphlab.SFrame('../../lecturenote/amazon_baby.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to ntngh2712@gmail.com and will expire on November 01, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-8328 - Server binary: /Users/nghiant/dato-env/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1447663155.log
[INFO] GraphLab Server Version: 1.6.1


## 1. Sums of words usage

In [6]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [7]:
products['adv_word_count'] = products['word_count'].apply(lambda x: advWordCount(x, selected_words))

In [8]:
graphlab.canvas.set_target('ipynb')

In [9]:
products['adv_word_count'].show()

## 2. Create new sentiment analysis model

In [10]:
#ignore all 3* reviews
products = products[products['rating'] != 3]
#positive sentiment = 4, 5* review
products['sentiment'] = products['rating'] >= 4
#split data set
train_data,test_data = products.random_split(.8, seed=0)

In [11]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['adv_word_count'],
                                                     validation_set=None)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+
PROGRESS: | 1         | 2        | 1.089179     | 0.844299          |
PROGRESS: | 2         | 3        | 1.151202     | 0.844186          |
PROGRESS: | 3         | 4        | 1.211780     | 0.844276          |
PROGRESS: | 4         | 5        | 1.272862     | 0.844269          |
PROGRESS: | 5         | 6        | 1.332389     | 0.844269          |
PROGRESS: | 6         | 7  

In [12]:
selected_words_model['coefficients'].sort('value', ascending=False).print_rows(num_rows=12)

+----------------+-----------+-------+------------------+
|      name      |   index   | class |      value       |
+----------------+-----------+-------+------------------+
| adv_word_count |    love   |   1   |  1.39989834302   |
|  (intercept)   |    None   |   1   |  1.36728315229   |
| adv_word_count |  awesome  |   1   |  1.05800888878   |
| adv_word_count |  amazing  |   1   |  0.892802422508  |
| adv_word_count | fantastic |   1   |  0.891303090304  |
| adv_word_count |   great   |   1   |  0.883937894898  |
| adv_word_count |    wow    |   1   | -0.0541450123333 |
| adv_word_count |    bad    |   1   | -0.985827369929  |
| adv_word_count |    hate   |   1   |  -1.40916406276  |
| adv_word_count |   awful   |   1   |  -1.76469955631  |
| adv_word_count |  horrible |   1   |  -1.99651800559  |
| adv_word_count |  terrible |   1   |  -2.09049998487  |
+----------------+-----------+-------+------------------+
[12 rows x 4 columns]



In [13]:
sentiment_model = graphlab.logistic_classifier.create(train_data, 
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=None)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 219217
PROGRESS: Number of coefficients    : 219218
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+
PROGRESS: | 1         | 5        | 0.000002  | 1.217478     | 0.841481          |
PROGRESS: | 2         | 9        | 3.000000  | 2.349490     | 0.947425          |
PROGRESS: | 3         | 10       | 3.000000  | 2.773504     | 0.923768          |
PROGRESS: | 4         | 11       | 3.000000  | 3.204233     | 0.971779          |
PROGRESS: | 

In [14]:
majority_classifier = graphlab.classifier.create(train_data,
                                                target='sentiment')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: LogisticClassifier, SVMClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.
PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 126811
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 5
PROGRESS: Number of unpacked features : 211906
PROGRESS: Number of coefficients    : 365037
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +--

## 3. Evaluation

In [15]:
selected_words_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+-------------------+-----+-------+------+
 |    threshold     |        fpr        | tpr |   p   |  n   |
 +------------------+-------------------+-----+-------+------+
 |       0.0        | 0.000187687687688 | 0.0 | 27976 | 5328 |
 | 0.0010000000475  |   0.999812312312  | 1.0 | 27976 | 5328 |
 | 0.00200000009499 |   0.999624624625  | 1.0 | 27976 | 5328 |
 | 0.00300000002608 |   0.999624624625  | 1.0 | 27976 | 5328 |
 | 0.00400000018999 |   0.999436936937  | 1.0 | 27976 | 5328 |
 | 0.00499999988824 |   0.999436936937  | 1.0 | 27976 | 5328 |
 | 0.00600000005215 |   0.999249249249  | 1.0 | 27976 | 5328 |
 | 0.00700000021607 |   0.999249249249  | 1.0 | 27976 | 5328 |
 | 0.00800000037998 |   0.999249249249  | 1.0 | 27976 | 5328 |
 | 0.00899999961257 |   0.999249249249  | 1.0 | 27976 | 5328 |
 +------------------+-------------------+-----+-------+------+
 [1001 rows x 

In [16]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+----------------+------------------+-------+------+
 |    threshold     |      fpr       |       tpr        |   p   |  n   |
 +------------------+----------------+------------------+-------+------+
 |       0.0        | 0.210655120482 | 0.00708159851963 | 28101 | 5312 |
 | 0.0010000000475  | 0.789344879518 |  0.99291840148   | 28101 | 5312 |
 | 0.00200000009499 | 0.751506024096 |  0.991317034981  | 28101 | 5312 |
 | 0.00300000002608 | 0.730609939759 |  0.990391801004  | 28101 | 5312 |
 | 0.00400000018999 | 0.715549698795 |  0.989822426248  | 28101 | 5312 |
 | 0.00499999988824 | 0.705195783133 |  0.989288637415  | 28101 | 5312 |
 | 0.00600000005215 | 0.695594879518 |  0.988683676737  | 28101 | 5312 |
 | 0.00700000021607 | 0.685993975904 |  0.988114301982  | 28101 | 5312 |
 | 0.00800000037998 | 0.676016566265 |  0.987794028682  | 28101 | 5312 |
 | 0.00899999961257 

In [17]:
majority_classifier.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+----------------+----------------+-------+------+
 |    threshold     |      fpr       |      tpr       |   p   |  n   |
 +------------------+----------------+----------------+-------+------+
 |       0.0        | 0.713168187744 | 0.068578375286 | 27968 | 5369 |
 | 0.0010000000475  | 0.286831812256 | 0.931421624714 | 27968 | 5369 |
 | 0.00200000009499 | 0.286831812256 | 0.931421624714 | 27968 | 5369 |
 | 0.00300000002608 | 0.286831812256 | 0.931421624714 | 27968 | 5369 |
 | 0.00400000018999 | 0.286831812256 | 0.931421624714 | 27968 | 5369 |
 | 0.00499999988824 | 0.286831812256 | 0.931421624714 | 27968 | 5369 |
 | 0.00600000005215 | 0.286831812256 | 0.931421624714 | 27968 | 5369 |
 | 0.00700000021607 | 0.286831812256 | 0.931421624714 | 27968 | 5369 |
 | 0.00800000037998 | 0.286831812256 | 0.931421624714 | 27968 | 5369 |
 | 0.00899999961257 | 0.286831812256 | 0.931

In [18]:
selected_words_model.show(view='Evaluation')
sentiment_model.show(view='Evaluation')
majority_classifier.show(view='Evaluation')

## 4. Interpreting

In [19]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']
diaper_champ_reviews.head()

name,review,rating,word_count,adv_word_count,sentiment
Baby Trend Diaper Champ,Ok - newsflash. Diapersare just smelly. We've ...,4.0,"{'son': 1, 'just': 2,'less': 1, '-': 3, ...",{},1
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1.0,"{'material)': 1, 'bags,':1, 'less': 1, 'when': 3, ...",{},0
Baby Trend Diaper Champ,Excellent diaper disposalunit. I used it in ...,5.0,"{'control': 1, 'am': 1,'it': 1, 'used': 1, ' ...",{},1
Baby Trend Diaper Champ,We love our diaper champ.It is very easy to use ...,5.0,"{'and': 3, 'over.': 1,'all': 1, 'bags.': 1, ...",{'love': 1},1
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'just': 1, '-': 3,'both': 1, 'results': 1, ...",{'horrible': 1},1
Baby Trend Diaper Champ,I waited to review thisuntil I saw how it ...,4.0,"{'lysol': 1, 'all': 1,'mom.': 1, 'busy': 1, ...",{'bad': 1},1
Baby Trend Diaper Champ,I have had a diaper geniefor almost 4 years since ...,1.0,"{'all': 1, 'bags.': 1,'just': 1, ""don't"": 2, ...",{},0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'lysol': 1, 'all': 2,'bags.': 1, 'feedback': ...",{},1
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5.0,"{'and': 2, 'all': 1,'just': 1, 'is': 2, ' ...",{},1
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'hand.': 1, 'both': 1,'(required': 1, 'befo ...",{'love': 2},1


In [20]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [21]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [22]:
diaper_champ_reviews[0]

{'adv_word_count': {},
 'name': 'Baby Trend Diaper Champ',
 'predicted_sentiment': 0.9999999372669541,
 'rating': 5.0,
 'review': 'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated in

In [23]:
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.7969408512906727]